In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import sys
import os
import pickle

from classes import boundaries as bd
plot_setup = bd.plot_setup()

%config Completer.use_jedi = False 




#--- Notebook run settings
run = 'command line' # 'manual' 'command line'
#run = 'manual'


# Settings for the multiple linear regression model (MLR)
#--- Arguments
if run == 'command line':
    # Read command line arguments
    arguments = os.environ['NB_ARGS']
    alpha, date_exp_str = arguments.split(',')
    
    alpha              = float(alpha)   
    
elif run == 'manual':
    # Enter arguments manually 
    alpha              = 0.01
    date_exp_str       = '230417_TempSS_FMort'
    remove_data        = True

print(date_exp_str, alpha)    

230417_TempSS_FMort 0.01


In [3]:
import glob
print(glob.glob('site_reg_results/'+date_exp_str+'*_SITE_best_models_dAIC.pkl'))
print(glob.glob('spe_reg_results/'+date_exp_str+'*_SPE_best_models_dAIC.pkl'))

['site_reg_results/230417_TempSS_FMort_0.01_SITE_best_models_dAIC.pkl']
['spe_reg_results/230417_TempSS_FMort_0.01_SPE_best_models_dAIC.pkl']


In [4]:
#---------------#
#  Import data  #
#---------------#
mtf_type = ['count','cmass']
mortality_groups = ['M_All','M_Fire','M_NoFire','M_Other']

with open('site_reg_results/'+date_exp_str+'_'+str(alpha)+'_SITE_best_models_dAIC.pkl','rb') as f:
    df_res_mort_site = pickle.load(f)
    
with open('spe_reg_results/'+date_exp_str+'_'+str(alpha)+'_SPE_best_models_dAIC.pkl','rb') as f:
    df_res_mort_spe = pickle.load(f)    

In [5]:
# Table B2
if 'TempSS_FMort' in date_exp_str:
    bd.print_header('Table B3')
else:
    bd.print_header('Table B2')
df_all_mort_levels = pd.concat([df_res_mort_site,df_res_mort_spe])
display(df_all_mort_levels)

with open(date_exp_str+'_TableB2_df.pkl','wb') as f:
    pickle.dump(df_all_mort_levels,f)


#--------------#
#   Table B3   #
#--------------#



r2         AIC       d_AIC  \
                                             count cmass count cmass count   
Level   Mortality model                                                      
Site    M_NoFire  MTF ~ MATsoil               0.24  0.29  91.1  93.5     0   
                  MTF ~ MAT                   0.21  0.25  92.6  95.9   1.5   
        M_Other   MTF ~ MATsoil               0.22  0.28  67.1  69.6     0   
                  MTF ~ MAT                    NaN  0.23   NaN  74.7   NaN   
Species M_NoFire  MTF ~ MAT + DBH_mean        0.36   0.4  87.7  93.5     0   
                  MTF ~ MATsoil + DBH_mean    0.35  0.37    89  96.1   1.3   
        M_Other   MTF ~ MAT + PFT + DBH_mean  0.43   NaN  78.8   NaN     0   
                  MTF ~ MATsoil + DBH_mean    0.36  0.39  83.3  88.9   4.5   
                  MTF ~ MAT + DBH_mean        0.35  0.38  84.4  89.7   5.6   

                                                    RMSE          ID        \
                                             cmass count cmass count cmass   
Level   Mortality model                                                      
Site    M_NoFire  MTF ~ MATsoil                  0  25.6  50.4     0     1   
                  MTF ~ MAT                    2.4  25.1  49.4     5     3   
        M_Other   MTF ~ MATsoil                  0  18.7  37.6     2     3   
                  MTF ~ MAT                    5.1   NaN  55.2   NaN     7   
Species M_NoFire  MTF ~ MAT + DBH_mean           0  12.0  18.1     1     1   
                  MTF ~ MATsoil + DBH_mean     2.6  12.3  18.6     2     2   
        M_Other   MTF ~ MAT + PFT + DBH_mean   NaN  11.4   NaN     0   NaN   
                  MTF ~ MATsoil + DBH_mean       0  12.7  19.2     2     1   
                  MTF ~ MAT + DBH_mean         0.8  12.4  18.8     3     3   

                                             N_obs       climate  
                                                                  
Level   Mortality model                                           
Site    M_NoFire  MTF ~ MATsoil               38.0  WorldClim30s  
                  MTF ~ MAT                   38.0  WorldClim30s  
        M_Other   MTF ~ MATsoil               29.0       CRUclim  
                  MTF ~ MAT                   28.0  WorldClim30s  
Species M_NoFire  MTF ~ MAT + DBH_mean        53.0     CRUNCEPv7  
                  MTF ~ MATsoil + DBH_mean    53.0     CRUNCEPv7  
        M_Other   MTF ~ MAT + PFT + DBH_mean  49.0     CRUNCEPv7  
                  MTF ~ MATsoil + DBH_mean    49.0     CRUNCEPv7  
                  MTF ~ MAT + DBH_mean        49.0     CRUNCEPv7

In [22]:
# Table 4
if 'TempSS_FMort' not in date_exp_str:
    bd.print_header('Table 4 - '+date_exp_str.split('_')[1])

    #df_all_mort_levels = df_all_mort_levels.reset_index().set_index(['Level','Mortality','model','climate']).copy()

    df_best_mort_model = pd.DataFrame()

    for t,tpe in enumerate(mtf_type):
        df_mtf_type = df_all_mort_levels.copy().swaplevel(axis=1)[tpe].reset_index()

        # Select model by mrotality group with the minimum RMSE when ∆RMSE < 0.5 years
        df_mtf_type['RMSE_min_diff'] = (df_mtf_type.RMSE - 
                                        df_mtf_type.groupby(['Level','Mortality'])['RMSE'].transform('min'))    
        df_min_rmse = df_mtf_type[df_mtf_type['RMSE_min_diff'] < 0.5]
        df_min_rmse.reset_index(inplace=True)

        # Otherwise choose largest variance explained
        df_max_r2 = df_min_rmse.groupby(['Level','Mortality'])['r2'].max().to_frame()
        max_r2_idx = [np.where(df_min_rmse.r2 == i) for i in df_max_r2.r2.tolist()]
        max_r2_idx_flat = [item for sublist in max_r2_idx for item in sublist]
        max_r2_idx_flat = [item for sublist in max_r2_idx_flat for item in sublist]
        df_best_mort = df_min_rmse.loc[df_min_rmse.index.isin(max_r2_idx_flat),:]

        # Format dataframe
        df_best_mort.set_index(['Level','Mortality','model'], inplace=True)
        df_best_mort = df_best_mort.drop('index',axis=1).copy()
        df_best_mort.columns = pd.MultiIndex.from_tuples([(i,tpe) for i in df_best_mort.columns])
        df_best_mort_model = pd.concat([df_best_mort_model,df_best_mort],axis=1)


    df_best_mort_model['N_obs']  = df_all_mort_levels.reset_index().groupby(['Level','Mortality'])['N_obs'].first()

    # If there are <20 observations set results to NaN (Management exclusion, Site, Fire mortality)
    #df_best_mort_model[df_best_mort_model.N_obs < 20] = np.nan

    df_best_mort_model = df_best_mort_model.reset_index().set_index(['Level','Mortality','model'])
    #df_best_mort_model['Climate'] = df_best_mort_model.climate 

    # Store result
    with open(date_exp_str+'_Table4_df.pkl','wb') as f:
        pickle.dump(df_best_mort_model[['r2','RMSE','ID','N_obs']],f)
    display(df_best_mort_model[['r2','RMSE','N_obs']])
    df_best_mort_model[['r2','RMSE','ID']]
    print(df_best_mort_model[['r2','RMSE','ID']].to_latex())


#--------------------------#
#   Table 4 - Management   #
#--------------------------#



r2        RMSE  \
                                                        count cmass count   
Level   Mortality model                                                     
Site    M_All     MTF ~ MAT + DBH_mean + Managed         0.36  0.42  23.4   
        M_Fire    MTF ~ DBH_mean                         0.54  0.47  21.0   
        M_NoFire  MTF ~ MAT                              0.32  0.37  23.0   
        M_Other   MTF ~ MAT                              0.36  0.43  24.4   
Species M_All     MTF ~ MAT + Humidity + PFT + DBH_mean  0.65  0.63  25.7   
        M_Fire    MTF ~ DBH_mean                          0.7  0.65  38.4   
        M_NoFire  MTF ~ MAT + PFT + DBH_mean             0.54  0.54  12.4   
        M_Other   MTF ~ MAT + PFT + DBH_mean             0.58  0.58  12.5   

                                                               N_obs  
                                                        cmass         
Level   Mortality model                                               
Site    M_All     MTF ~ MAT + DBH_mean + Managed         49.2   64.0  
        M_Fire    MTF ~ DBH_mean                         54.9   17.0  
        M_NoFire  MTF ~ MAT                              45.0   47.0  
        M_Other   MTF ~ MAT                              47.8   37.0  
Species M_All     MTF ~ MAT + Humidity + PFT + DBH_mean  43.6  107.0  
        M_Fire    MTF ~ DBH_mean                         67.8   37.0  
        M_NoFire  MTF ~ MAT + PFT + DBH_mean             18.8   70.0  
        M_Other   MTF ~ MAT + PFT + DBH_mean             19.0   61.0

\begin{tabular}{lllllrrll}
\toprule
        &         &                            & \multicolumn{2}{l}{r2} & \multicolumn{2}{l}{RMSE} & \multicolumn{2}{l}{ID} \\
        &         &                            & count & cmass & count & cmass & count & cmass \\
Level & Mortality & model &       &       &       &       &       &       \\
\midrule
Site & M\_All & MTF \textasciitilde  MAT + DBH\_mean + Managed &  0.36 &  0.42 &  23.4 &  49.2 &     0 &     0 \\
        & M\_Fire & MTF \textasciitilde  DBH\_mean &  0.54 &  0.47 &  21.0 &  54.9 &     0 &     0 \\
        & M\_NoFire & MTF \textasciitilde  MAT &  0.32 &  0.37 &  23.0 &  45.0 &    26 &    31 \\
        & M\_Other & MTF \textasciitilde  MAT &  0.36 &  0.43 &  24.4 &  47.8 &    35 &    34 \\
Species & M\_All & MTF \textasciitilde  MAT + Humidity + PFT + DBH\_mean &  0.65 &  0.63 &  25.7 &  43.6 &     1 &     1 \\
        & M\_Fire & MTF \textasciitilde  DBH\_mean &   0.7 &  0.65 &  38.4 &  67.8 &     1 &     1 \\
        & M\_NoF